In [1]:
# Learning LHS sampling

import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Next section is examples of creating LHS from list of variables. Take, for instance, an eight-layer RTS laminate.
# [pm phi_1 < T0_1 | T1_1 > n_1 , pm phi_2 < T0_2 | T1_2 > n_2]s such that there are 8 variables

# x = [phi1, t01, t11, n1, phi2, t02, t12, n2]

# as n_i is dependent on phi_i, normalise n_i to be between 0 and 1 and then, dependent on phi, define integer value

# total number of samples for an eight-layer RTS laminate (assuming symmetry and balance):
# n_phi=0 = [0, 1, ..., 10] therefore when phi = 0, number of variables for n = 11
# n_phi=90 [0, 1, ..., 18] therefore when phi = 90, number of variables for n = 18
# t0 = t1 = [0, 1, ..., 70] therefore number of variables = 71
# each cylinder (of eight layers) is defined by two RTS lamina: x = [phi1, t01, t11, n1, phi2, t02, t12, n2]

# total number of samples (phi = 0) + (phi = 0 and 90) + (phi = 90)
n_tot = (71**4*11**2) + 2*(71**4*11*19) + (71**4*19**2)
print(f'Total number of variables = {n_tot}')

Total number of variables = 22870512900


# Create LHS samples

In [3]:
def rts_lhs():

    # defining function to take LHS (n_samples-by-n_variables matrix of value 0 -> 1) into useable designs
    def lhs_to_lamina(lhs_in, cyl, lam_type):

        import numpy as np

        # lhs_in = unity-normalised LHS
        # cyl = [radius, length, thickness]
        # lam_type = 1 (RTS) or lam_type = 0 (SF)

        # return = lamina = [[phi1, t01, t11, n1, phi2, t02, t12, n2]*n_var]

        # define sub-function that takes phi and returns n_max based on geometry
        def phi_to_n_max(cyl, phi):

            # cyl = [radius, length, thickness]
            # phi = 0 (axial) or = 90 (circumferential)

            # assume MSR (minimum steering radius) is 50, therefore 1 period is 100 mm (minimum)
            msr = 100

            if phi == 0:
                n_max = np.floor(cyl[1]//msr) + 1
            elif phi == 90:
                n_max = np.floor(2*np.pi*cyl[0]//msr) + 1
            else:
                SyntaxError('Phi is not defined as 0 or 90 degree')

            return n_max

        lamina = np.zeros((np.shape(lhs_in)))

        # if lam_type = 1 (i.e. RTS laminate)
        if lam_type:

            for i, l in enumerate(lhs_in):
                lamina[i, 0] = round(l[0])*90 # defining phi_1
                lamina[i, 1] = round(l[1]*70) # defining t0_1
                lamina[i, 2] = round(l[2]*70) # defining t1_1
                lamina[i, 3] = int(l[3]*phi_to_n_max(cyl, lamina[i,0])) # defining n_1
                lamina[i, 4] = round(l[4])*90 # defining phi_2
                lamina[i, 5] = round(l[5]*70) # defining t0_2
                lamina[i, 6] = round(l[6]*70) # defining t1_2
                lamina[i, 7] = int(l[7]*phi_to_n_max(cyl, lamina[i,4])) # defining n_2

        # if lam_type = 0 (i.e. SF laminate)
        elif not lam_type:

            for i, l in enumerate(lhs_in):
                lamina[i, 0] = round(l[0]*90) # defining alpha_1
                lamina[i, 1] = round(l[1]*90) # defining alpha_2

        return np.asarray(lamina)

    ## Rapid tow sheared (RTS) laminates
    # number of RTS variables: [pm phi_1 < T0_1 | T1_1 > n_1 , pm phi_2 < T0_2 | T1_2 > n_2]s
    n_rts_var = 8

    # defining number of samples for RTS laminates
    n_rts_sam = 10*n_rts_var
    # the number of samples is based on the 'levels' of each variable:
    # # https://stats.stackexchange.com/questions/58201/how-to-determine-the-sample-size-of-a-latin-hypercube-sampling
    # SMT use 10 * n dimensions. We have 8 dimensions, therefore 80 samples. Could change... Probably need to infill after initial 

    from pyDOE2 import lhs

    # call LHS from pyDOE2 for RTS laminates
    lhs_rts  = lhs(n_rts_var, n_rts_sam)

    # cyl is a vector with [radius, length, thickness] in mm of the cylinder in question
    # cyl is needed to define max periodicty if RTS laminate is chosen
    cyl = [300, 1040, 1.05]

    # transform unity lhs to laminate definition for RTS (lam_type = 1)
    rts_lams = lhs_to_lamina(lhs_rts, cyl, 1)

    return rts_lams

rts_lams = rts_lhs()

# # transform unity lhs to laminate definition for SF (lam_type = 0)
# sf_lams = lhs_to_lamina(lhs_sf, cyl, 0)

# print(rts_lams[0:4])

# print(sf_lams)

# Run LHS samples through "Abaqus"


In [4]:
# Run LHS with Abaqus. For now, use dummy function where y (output) is the sum of the variables
def dummy_func(x_in):
    
    y_out = []
    
    for x in x_in:
        y_out.append(sum(x) + sum(x)/np.sqrt(sum(x)) + np.sin(x[1]))
    
    return np.asarray(y_out)

y_rts = dummy_func(rts_lams)
print(y_rts[0:4])
# y_sf = dummy_func(sf_lams)

[153.71581307  48.55744832 141.35068934 171.98547676]


# Calculate convergence crit

In [5]:
from smt.surrogate_models import KRG

def split_data(data_in, data_out, train_per, test_per):
    
    import numpy as np
    from random import sample
    from random import seed
    
    # set seed 
    seed(1)
    
    # define number of data entries
    data_nums = np.arange(0, len(data_in)).tolist()
    
    # get data indicies that will be trained on (train_per %) by randomly sampling (with set seed)
    train_nums = np.sort(sample(data_nums, round(len(data_in)*train_per/100)))
    
    # keep back data indicies that will be tested on (test_per %)
    test_nums = np.asarray([x for x in data_nums if x not in train_nums])
    
    # putting training data together
    train_in = data_in[train_nums[:]]
    train_out = np.atleast_2d(data_out[train_nums[:]]).T
    train_data = np.concatenate((train_in,train_out),axis=1)
    
    # putting testing data together
    test_in = data_in[test_nums[:]]
    test_out = np.atleast_2d(data_out[test_nums[:]]).T
    test_data = np.concatenate((test_in, test_out),axis=1)

    # check data_in = train_in + test_in and data_out = train_out + test_out
    if len(data_in) != len(train_in) + len(test_in):
        SyntaxError('Length of train_in + test_in is not the same as data_in: data has been missed/added?!')
    if len(data_out) != len(train_out) + len(test_out):
        SyntaxError('Length of train_out + test_out is not the same as data_out: data has been missed/added?!')
    
    return train_data, test_data

rts_train, rts_test = split_data(rts_lams, y_rts, 80, 20)
# sf_train, sf_test = split_data(sf_lams, y_sf, 80, 20)

# RTS laminates
sm_rts = KRG(theta0=[1e-1])
sm_rts.set_training_values(rts_train[:,0:8], rts_train[:,8])
sm_rts.train()

rts_pred = sm_rts.predict_values(rts_test[:,0:8])
print(rts_pred)
print(rts_test[:,8])

# estimated variance
# s2 = sm.predict_variances(x)
# derivative according to the first variable
# dydx = sm.predict_derivatives(xt, 0)
# fig, axs = plt.subplots(1)

___________________________________________________________________________
   
                                  Kriging
___________________________________________________________________________
   
 Problem size
   
      # training points.        : 64
   
___________________________________________________________________________
   
 Training
   
   Training ...
   Training - done. Time (sec):  0.2993660
___________________________________________________________________________
   
 Evaluation
   
      # eval points. : 16
   
   Predicting ...
   Predicting - done. Time (sec):  0.0010026
   
   Prediction time/pt. (sec) :  0.0000627
   
[[142.64540374]
 [192.14491005]
 [415.61008792]
 [288.6308756 ]
 [196.44573248]
 [140.19795701]
 [224.73962105]
 [167.06057359]
 [338.40215654]
 [336.143444  ]
 [184.08918978]
 [155.53355461]
 [311.2823489 ]
 [330.42802524]
 [259.17168   ]
 [298.98867917]]
[141.35068934 192.18313505 414.35305591 285.57615675 197.44069451
 139.95377905 226.482214